In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from utils import unpickle
import pandas as pd

%load_ext autoreload
%autoreload 2
%load_ext tensorboard
# %tensorboard --logdir logs

plt.style.use("lusk")

In [ ]:
ignore_stats = np.load("stats/10folds_CV_ignore_label.npy", allow_pickle=True)
loose_stats = np.load("stats/10folds_CV_loose.npy")
strict_stats = np.load("stats/10folds_CV_strict.npy")
simp_imp_stats = np.load("stats/10folds_CV_simp_imp.npy", allow_pickle=True)

In [ ]:
df = pd.DataFrame(np.c_[np.repeat("Unaltered", len(loose_stats)), loose_stats], columns=["IoU Type", "Overall bIoU", "Tree bIoU", "BG bIoU"])
df = df.append(pd.DataFrame(np.c_[np.repeat("Impute (channel mean)", len(strict_stats)), strict_stats], columns=df.columns), ignore_index=True)
df = df.append(pd.DataFrame(np.c_[np.repeat("Impute (column mean)", len(simp_imp_stats[:, -3:])), simp_imp_stats[:, -3:]], columns=df.columns), ignore_index=True)
df = df.append(pd.DataFrame(np.c_[np.repeat("Ignore", len(ignore_stats[:, -3:])), ignore_stats[:, -3:]], columns=df.columns), ignore_index=True)

for i in range(1, 4):
    df.iloc[:, i] = df.iloc[:, i].astype(np.float32)
    
pd.options.display.float_format = '{:.3f}'.format

In [ ]:
from rasterio import pad


titles = ["Overall bIoU", "Tree bIoU", "BG bIoU"]

_, ax = plt.subplots(1, 1, figsize=(10, 4), dpi=300)
width = 0.15
spacing = 0.155
r = np.arange(3)

errs = {"elinewidth": 1, "capsize": 2}

ax.bar(
    r - 1.5 * spacing,
    loose_stats.mean(axis=0),
    width,
    label="Include loose",
    yerr=loose_stats.std(axis=0),
    error_kw=errs,
)
ax.bar(
    r - 0.5 * spacing,
    strict_stats.mean(axis=0),
    width,
    label="Impute loose (channel mean)",
    yerr=strict_stats.std(axis=0),
    error_kw=errs,
)
ax.bar(
    r + 0.5 * spacing,
    simp_imp_stats[:, -3:].mean(axis=0),
    width,
    label="Impute loose (column mean)",
    yerr=simp_imp_stats[:, -3:].astype(float).std(axis=0),
    error_kw=errs,
)
ax.bar(
    r + 1.5 * spacing,
    ignore_stats[:, -3:].mean(axis=0),
    width,
    label="Ignore loose",
    yerr=ignore_stats[:, -3:].astype(float).std(axis=0),
    error_kw=errs,
)
ax.set_xticks(r, titles)
ax.set_xlabel("IoU Type")
ax.set_ylabel("Binary IoU")
ax.grid()
ax.legend(fontsize=10, loc="upper center")
ax.set_ylim(0.2, 0.95)

In [ ]:
loss = unpickle(ignore_stats[:, 0])
val_loss = unpickle(ignore_stats[:, 1])
iou = unpickle(ignore_stats[:, 2])
val_iou = unpickle(ignore_stats[:, 3])
epochs = np.arange(0, loss.shape[-1], 1)

_, (ax0, ax1) = plt.subplots(1, 2, figsize=(18, 7), dpi=200)

ax0.plot(epochs, loss.mean(axis=0), "y", label="Training loss")
ax0.plot(epochs, val_loss.mean(axis=0), "r--", label="Validation loss")
ax0.set_xlabel("Epochs")
ax0.set_ylabel("Loss")
ax0.legend()
ax0.grid()

ax1.plot(epochs, iou.mean(axis=0), "y", label="Training IoU")
ax1.plot(epochs, val_iou.mean(axis=0), "r--", label="Validation IoU")
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Loss")
ax1.legend()
ax1.grid()
# ax1.set_ylim(0.492, 0.493)
    
# fg, ax = plt.subplots(1, 2, figsize=(11, 5), dpi=170)
# ax[0].plot(epochs, loss, "y", label="Training loss")
# ax[0].plot(epochs, val_loss, "r", label="Validation loss")
# ax[0].set_title("Training and validation loss")
# ax[0].set_xlabel("Epochs")
# ax[0].set_ylabel("Loss")
# ax[0].legend()

# ax[1].plot(epochs, acc, "y", label="Training acc")
# ax[1].plot(epochs, val_acc, "r", label="Validation acc")
# ax[1].set_title("Training and validation accuracy")
# ax[1].set_xlabel("Epochs")
# ax[1].set_ylabel("Accuracy")
# ax[1].legend()
plt.show()